In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#  Extraccion de datos 
df =  pd.read_csv("../data/sharks-clean.csv", encoding="latin1")

#### ¿Se puede sobrevivir al ataque de un tiburon ? 

In [3]:
df['Fatal_(Y/N)'].value_counts()

N          4301
Y          1389
Unknown      71
2017          1
M             1
Name: Fatal_(Y/N), dtype: int64

 Filtramos el data frame solo por supervivientes.


In [4]:
live_df =df[ df['Fatal_(Y/N)'] == 'N' ]

In [5]:
live_df.head()

,Unnamed: 0,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_(Y/N),Time,Species
0,0,2018.06.25,25-Jun-2018,2018.0,Boating,Usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No Injury To Occupant, Outrigger Canoe And Pad...",N,18H00,White Shark
1,1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,Usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson Mcneely,F,11,Minor Injury To Left Thigh,N,14H00 -15H00,NaN
2,2,2018.06.09,09-Jun-2018,2018.0,Invalid,Usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury To Left Lower Leg From Surfboard Skeg,N,07H45,NaN
3,3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,Australia,New South Wales,Arrawarra Headland,Surfing,Male,M,NaN,Minor Injury To Lower Leg,N,NaN,2 M Shark
4,4,2018.06.04,04-Jun-2018,2018.0,Provoked,Mexico,Colima,La Ticla,Free Diving,Gustavo Ramos,M,NaN,Lacerations To Leg & Hand Shark Provoked Incident,N,NaN,"Tiger Shark, 3M"


In [6]:
# Contabilizar el numero de supervivientes 

len(live_df )

4301

In [7]:
#Total de muestras: al tener datos que generan ruido nos quedamos unicamente con los afirmativo y negtivos.
Muestra = len(live_df)+ len(df['Fatal_(Y/N)'] == 'Y')
Muestra

10605

In [8]:
f'la probabilidad de supervivencia tras ser atacado por un tiburon es del {round(100*len(live_df)/Muestra,2)}%'

'la probabilidad de supervivencia tras ser atacado por un tiburon es del 40.56%'

¿Cual es el pais con mayor supervivencia tras el ataque de un tiburon?

In [9]:
country_df = live_df.Country.value_counts()

In [10]:
# % de ataques por pais
plive_df = country_df.apply(lambda x:f'{round(100*x/country_df.sum(),2)}%')
plive_df

Usa                   42.9%
Australia            21.51%
South Africa           9.5%
Bahamas                2.2%
New Zealand           2.13%
                      ...  
Curacao               0.02%
Mediterranean Sea     0.02%
Egypt / Israel        0.02%
Iran / Iraq           0.02%
Belize                0.02%
Name: Country, Length: 146, dtype: object

In [11]:
# Buscamos el area mas segura para sobrevivir
live_df [['Country', 'Area']]['Area'].value_counts()

Florida                 918
New South Wales         334
California              242
Hawaii                  213
Queensland              210
                       ... 
Red Sea Protectorate      1
Fife                      1
Woodlark Islands          1
Santa Catarina State      1
Vera Cruz                 1
Name: Area, Length: 510, dtype: int64

¿Que especie mata mas ?

In [12]:
# Supervivencia ante los 5 mas agresivos   
supervivencia_agresivas = live_df.Species.value_counts()
supervivencia_agresivas.iloc[:5]


White Shark    116
Tiger Shark     49
4' Shark        40
Bull Shark      38
6' Shark        38
Name: Species, dtype: int64

In [13]:
# mortalidad ante los 5 mas agresivos 
dead_df =df[ df['Fatal_(Y/N)'] == 'Y' ]
mortalidad_agresivas = dead_df.Species.value_counts()
mortalidad_agresivas.iloc[:5]

White Shark          44
Tiger Shark          25
Bull Shark           16
3.7 M [12'] Shark     9
3 M [10'] Shark       8
Name: Species, dtype: int64

Parece que tanto el tiburon tigre como el gran blanco son los mas agresivos ...¿atacaran de de manera intencionada?

In [14]:
# generamos un df con la palabra clave correspondiente por si ubiese algun registro mal tipeado
white_df = df[df['Species'].str.contains('White', na=False, regex=False)]

tigre_df = df[df['Species'].str.contains('Tiger', na=False, regex=False)]



In [15]:
w_attack_df = white_df['Type'].value_counts().head()
w_attack_df

Unprovoked    505
Boating        67
Boat           56
Provoked       26
Invalid         7
Name: Type, dtype: int64

In [16]:
tigre_df['Type'].value_counts().head()

Unprovoked      225
Provoked         15
Sea Disaster     12
Boating           7
Boat              3
Name: Type, dtype: int64

A primera vista parece que hambos realizan los ataques sin una provocacion previa aunque el gran blanco destaca por su agresividad atacando a objetos en superficie y claramente no es recomendable provocarle. 

¿Que Actividad suelen practicar las personas que sufren el ataque del gran Blanco en su Pais?

In [36]:
abp = white_df.groupby(by=['Country','Activity']).agg(act_count=('Activity','count'))

In [37]:
abp.sort_values("act_count",ascending=False).iloc[:10]

act_count
Country      Activity                
Usa          Surfing               63
Australia    Surfing               42
South Africa Surfing               39
             Spearfishing          22
Australia    Fishing               19
South Africa Swimming              18
Usa          Kayaking              16
             Swimming              12
South Africa Body Boarding         12
Australia    Swimming              11

# Meme para tiburones

In [26]:
duplicados_df = df[df['Activity'].duplicated(keep=False)]['Activity'].tolist()
set_duplicados = set(duplicados_df)
set_duplicados

{'"Flying Tiger" Transport Plane Went Down With 5 Men Onboard',
 '.',
 'Air Disaster',
 'Aircraft Crashed Into Sea',
 'Attempting To Kill A Shark With Explosives',
 'Bathing',
 'Bathing In River',
 'Beach Seine Netting',
 'Boat',
 'Boat Capsized',
 'Boat Capsized, Swimming Ashore',
 'Boat Swamped',
 'Boat With 5 Men Capsized While Returning To The Amerique',
 'Boating',
 'Body Boarding',
 'Body Surfing',
 'Body Surfing / Treading Water',
 'Body-Boarding',
 'Boogie Boarding',
 'Cage Diving',
 'Canoeing',
 'Clamming',
 'Cleaning Fish',
 'Collecting Shells',
 'Competing In The Woodvale Atlantic Rowing Race',
 'Crabbing',
 'Crayfishing',
 'Cruising',
 'Dangling Feet In The Water',
 'Dived For A Coin',
 'Diving',
 'Diving Around Anchored Liner',
 'Diving For Abalone',
 'Diving For Beche-De-Mer',
 'Diving For Coins',
 'Diving For Crayfish',
 'Diving For Lobsters',
 'Diving For Sponges',
 'Diving For Trochus',
 'Diving For Trochus Shell',
 'Diving From Canoe',
 'Diving Into Water',
 'Diving O

In [31]:
# Muerte por escapar de la prision 
df[df['Activity'].str.contains('Imprisonment', na=False, regex=False)]


,Unnamed: 0,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_(Y/N),Time,Species
5784,5784,1883.09.14.R,Reported 14-Sep-1883 (Probably Happened Ca. 18...,1883.0,Unprovoked,Australia,Tasmania,Between Port Arthur Penal Colony & Forestier P...,Swimming / Escaping Imprisonment,Owen,M,NaN,Fatal,Y,NaN,NaN
5893,5893,1873.00.00,Nov- Or Dec-1873,1873.0,Unprovoked,Cuba,Havana Province,Havana Harbor,Swimming / Escaping Imprisonment,Franz Mayer,M,27,"Legs Severed Bitten, Later Surgically Amputated",N,NaN,NaN


In [23]:
#Muerte por accidente aereo y quizas comido por un tiburon 
df[df['Activity'].str.contains('Air', na=False, regex=False)]

,Unnamed: 0,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_(Y/N),Time,Species
504,504,2014.09.00,Sep-2014,2014.0,Unprovoked,Spain,Catalonia,Salou,Playing With An Air Mattress,Male,M,16,Lacerations To Right Hand,N,NaN,NaN
1158,1158,2009.02.08,08-Feb-2009,2009.0,Sea Disaster,Usa,Puerto Rico,Quebradillas,Air Disaster,Occupant Of A Cessna 206,M,NaN,It Is Probable That All 5 Passengers Died On I...,Y,NaN,NaN
1708,1708,2004.01.03.A,03-Jan-2004,2004.0,Sea Disaster,Egypt,Sinai Peninsula,Off Sharm El-Sheikh,Air Disaster. Flash Airlines Boeing 737 Crashe...,135 Passengers & 13 Crew,NaN,NaN,"No Survivors, Sharks Scavenged On Remains",Y,03H00,NaN
1865,1865,2002.04.30,30-Apr-2002,2002.0,Unprovoked,Australia,South Australia,"Smoky Bay, Near Ceduna, On The Eyre Peninsula",Scallop Diving (Using Surface-Supplied Air & A...,Paul Buckland,M,23,"Fatal, Torso & Leg Bitten",Y,12H40,"White Shark, 6M [20']"
2054,2054,2000.05.13,13-May-2000,2000.0,Sea Disaster,New Caledonia,South Province,Mont Dore,Air Disaster - Piper Aircraft Crashed Into The...,3 People,NaN,NaN,Sharks Prevented Recovery Of Remains,Y,NaN,Tiger Sharks & Bull Sharks (20 Sharks In All)
2331,2331,1995.11.25.A,25-Nov-1995,1995.0,Unprovoked,Northern Arabian Sea,NaN,NaN,Fell Off Aircraft Carrier,"Zacharay Mayo, A U.S. Marine",M,NaN,"Minor Injuries, Bites On Fingers & Toes. Rescu...",N,NaN,NaN
2933,2933,1983.07.05,05-Jul-1983,1983.0,Unprovoked,Usa,Florida,"Riviera Beach, Palm Beach County",Floating On Air Mattress,Steven Osterman,M,17,"No Injury, Shark Bit Air Mattress, Deflating It",N,NaN,NaN
3133,3133,1977.12.31,31-Dec-1977,1977.0,Sea Disaster,Usa,Hawaii,2 Miles Off Keahole Airport,"Swimming, After Single-Engine Aircraft Went Do...",Harold Corbett,M,49,Feet Lacerated,N,Night,NaN
3404,3404,1970.01.16,16-Jan-1970,1970.0,Sea Disaster,Egypt,Red Sea,A Few Miles South Of Port Suez,Air Disaster,Pilot Of Israeli Skyhawk,M,NaN,Fatal,Y,NaN,NaN
3591,3591,1966.01.14,14-Jan-1966,1966.0,Sea Disaster,Columbia,NaN,Off Cartagena,Colombian (Avianca) Dc-4 Airliner Plunged Int...,NaN,NaN,NaN,"10 Survived, 51 Perished.",Y,NaN,Shark Involvement Not Confirmed


In [33]:
# ataques mirandose los pies 
df[df['Activity'].str.contains('Washing His Feet', na=False, regex=False)]


,Unnamed: 0,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_(Y/N),Time,Species
1105,1105,2009.07.24.R,Reported 24-Jul-2009,2009.0,Unprovoked,Kenya,Mombasa,Likoni Channel,Washing His Feet,Male,M,NaN,Fatal,Y,NaN,NaN
6077,6077,1837.01.17,17-Jan-1837,1837.0,Unprovoked,Australia,New South Wales,Macleay River,Washing His Feet,Alfred Australia Howe,M,12,"Fatal Injured By Shark, Died Of Tetanus",Y,Evening,NaN


In [35]:
# si te tiran por la borda 

df[df['Activity'].str.contains('Thrown Overboard', na=False, regex=False)]

,Unnamed: 0,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_(Y/N),Time,Species
5696,5696,1890.12.27.R,Reported 27-Dec-1890,1890.0,Unprovoked,British New Guinea,Woodlark Islands,Off Panamota,Thrown Overboard,Tetebara,M,18,Arm Bitten,N,NaN,NaN
5697,5697,1890.10.25.R,Reported 25-Oct-1890,1890.0,Unprovoked,Atlantic Ocean,NaN,NaN,Thrown Overboard,Captain Lavarello,M,NaN,Fatal,Y,NaN,NaN
5803,5803,1882.00.00.B,1882,1882.0,Unprovoked,Usa,Florida,"In The Bay Near The Naval Yard At Pensacola, E...","During ""An Exhibition"" He Was Tied In Sack & T...",John T. Clark,M,NaN,"No Injury, Sack Rammed By Shark & Shark Haras...",N,NaN,NaN
6068,6068,1840.07.22.R,Reported 22-Jul-1840,1840.0,Unprovoked,Usa,South Carolina,"Charleston Harbor, Charleston County",Accidentally Thrown Overboard & Treading Water...,Crew Member Of A Pilot Boat,M,NaN,"Fatal, Body Was Not Recovered",Y,NaN,Said To Be A 7.6 M [25'] Shark
